![figure](../lab10/lab10_figures/politecnico_h-01.png)
# **Eletrónica Configurável / Configurable Electronics**
#### Mestrado em Engenharia Eletrotécnica / Master in Electrical and Electronic Engineering

## **LabWork10 -  Custom Hardware Accelerators**

______________

## Introduction ##
This tutorial demonstrates how to accelerate a Python function on the Zynq-7000 using PYNQ. We begin by testing a software FIR implementation, the **SciPy** function **lfilter**, and measuring it's performance. Then we load a custom overlay with a hardware FIR, and compare it's performance to the software implementation.


### Objectives ###
After completing this lab, you will be able to:

* Create Overlays in Vivado with hardware accelerators;
* Create accelerators with AXI streaming interface;
* Use AXI DMAs to interface with accelerators;
* Create APIs in Python for your accelerators;
* Transfer data between PS and PL;


In the instructions below **{sources}** refers to `C:\Xilinx\MEE_EC\sources` and **{labs}** refers to `(C:\Xilinx\MEE_EC\labs)`

This tutorial is inspired in [Jeff Johnson](https://www.fpgadeveloper.com/2018/03/how-to-accelerate-a-python-function-with-pynq.html/) video and [repository](https://gist.github.com/fpgadeveloper/c086319971f68226b424bce98d430098).

____

## Step 1  - Acceleration of a FIR filter on the PYNQ-Z2##

### Step 1.1 ###

In this first step you will Test/benchmark the FIR filter function in Python. we will star by defining the plot function, then generate the input signal and finally filter this signal using a software defined filter.

* Power-up your PYNQ board, connect it to the network and wait for linux to boot (LEDs will flash when it is ready to run).


* Open your web browser and connect to the Jupyter notebook server. Upload this lab10 notebook and run it from there.


* The first code block below defines a function that we will use for plotting data throughout this notebook. Note that the function has a **n_samples** argument so that we can limit the number of samples to plot. Plotting more than a few thousand samples can be very slow and consume a lot of RAM.

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

def plot_to_notebook(time_sec,in_signal,n_samples,out_signal=None):
    plt.figure()
    plt.subplot(1, 1, 1)
    plt.xlabel('Time (usec)')
    plt.grid()
    plt.plot(time_sec[:n_samples]*1e6,in_signal[:n_samples],'y-',label='Input signal')
    if out_signal is not None:
        plt.plot(time_sec[:n_samples]*1e6,out_signal[:n_samples],'g-',linewidth=2,label='FIR output')
    plt.legend()

* The next code block will generate a "noisy" signal that we will use as input to the FIR implementations. The main signal is a 200kHz sinusoid but we have added lower amplitude sinusoids at 46MHz and 12MHz to simulate noise.

In [ ]:
import numpy as np

# Total time
T = 0.002
# Sampling frequency
fs = 100e6
# Number of samples
n = int(T * fs)
# Time vector in seconds
t = np.linspace(0, T, n, endpoint=False)
# Samples of the signal
samples = 10000*np.sin(0.2e6*2*np.pi*t) + 1500*np.cos(46e6*2*np.pi*t) + 2000*np.sin(12e6*2*np.pi*t)
# Convert samples to 32-bit integers
samples = samples.astype(np.int32)
print('Number of samples: ',len(samples))

# Plot signal to the notebook
plot_to_notebook(t,samples,1000)

* Now we use the **lfilter** function from **SciPy** to filter the signal using our filter coefficients. The coefficients implement a low pass filter with passband 0-5MHz, and they were generated using [this website](http://t-filter.engineerjs.com/)

In [ ]:
from scipy.signal import lfilter

coeffs = [-255,-260,-312,-288,-144,153,616,1233,1963,2739,3474,4081,4481,4620,4481,4081,3474,2739,1963,1233,616,153,-144,-288,-312,-260,-255]

import time
start_time = time.time()
sw_fir_output = lfilter(coeffs,70e3,samples)
stop_time = time.time()
sw_exec_time = stop_time - start_time
print('Software FIR execution time: ',sw_exec_time)

# Plot the result to notebook
plot_to_notebook(t,samples,1000,out_signal=sw_fir_output)

Note that the output signal is a filtered sinusoid (filter works!) with a small phase shift (corresponds to the filter's latency). We can see that the software FIR execution time was about 0.1 seconds. In the next step we will try to speed that up.

### Step 1.2 ###

In this step we will design an overlay with a FIR hardware accelerator IP in Vivado.


* **Open** Vivado and create a new Vivado RTL project in **{labs}** with name **lab10**. Do not specify sources at this time and choose the PYNQ-Z2 board as the default part. Click **Finish**.


* Select **Create Block Design** and name it **lab10bd**, local to project. Click **OK**.


* Click **Add IP** and add a **ZYNQ7 Processing System** to the design canvas.


* **Run Block Automation** with all the default options. Click **OK**.


* **Double-click** the ZYNQ7 Processing System to configure the IP. We will have to enable a high performance port to the DDR because the DMA will need to access the DDR. In the **PS-PL Configuration** tab, click the **HP Slave AXI Interface** and enable one of them (e.g., **S AXI HP0 interface**). This will enable you to connect the DMA through to the DDR. Click **OK**.
    
![Figure](../lab10/lab10_figures/fig1.png)


* Back in the Block design, click **Add IP** again and add a **FIR Compiler** to the design canvas.


* **Double-click** the FIR Compiler to configure the IP. We will use the same coeficients as we used in the software filter, so you can copy them from the code cell above an paste them in the **Coefficient Vector** field, in **Filter Options** tab (just the comma separated values - no parenthesis).


* In **Channel Specification** tab, configure both the **Input Sampling Frequency** and **Clock Frequency** as **100** MHz. This way, the IP core will have a fully parallel implementation - the core will take a new sample at every clock period. As the DMA will be clocked at the same speed, the core will take every sample the DMA gives it.


* In **Implementation** tab, configure the **Input Data Width** to **32** bits. We want an output with 32 bits as well, so we will use an *Output Rounding Mode* equal to **Non-Symmetric rounding up** and specify the **Output Width** as **32** bits.


* Because we are going to push the input signal through this IP core using an AXI DMA, this core will have to have an AXI Streaming interface. So go to the **Interface** tab and enable both the **TLAST** and **TREADY** signals. In the TLAST drop-down menu select **Packet Framing** and make sure to **check** the *Output TREADY* box.


* Click **OK** to exit the FIR Compiler customization window.


* Now you should be back to the block design window. Click **Add IP** and add a **AXI Direct Memory Access** to the design canvas.


*  **Double-click** the AXI DMA to configure the IP. Make the following customization and click **OK** when finished.
    * **Disable** the Scatter Gatther Engine (not supported by PYNQ)
    * Maximize the **Width of Buffer Length Register** because this determines the maximum DMA transfer size (set to **26**).
    * Make sure that both **Read and Write Channels** are enabled, and data width is 32 bits.
    

* In the Block Design, drag the FIR filter next to the AXI DMA IP (so you can zoom-in and clearly see both modules). Make the following connections:
    * The streaming output of the FIR Compiler (**M_AXIS_DATA**) to the streaming input of the DMA (**S_AXIS_S2MM**).
    * The streaming output of the AXI DMA (**M_AXIS_MM2S**) to the input of the FIR Compiler (**S_AXIS_DATA).
    
![Figure](../lab10/lab10_figures/fig2.png)


* For the rest of the connections, just **Run Block Automation**. Select all the automation but inspect them one by one. Select an Interface on the left to display its configuration options on the right. This connection automation feature will:
    * Connect the DMA slave interface (**/axi_dma_0/S_AXI_Lite**) to the processing system (PS) general purpose AXI interface (**M_AXI_GP0**).
    * Connect the clock of the FIR Compiler to the fabric clock zero (**FCLK_CLK0**) which is 100MHz as we desire.
    * Connect the PS high performance port (**/processing_system7_0/S_AXI_HP0**) to one of the memory map ports of the DMA (**/axi_dma_0/M_AXI_MM2S**).


* Click **OK** to run the connection automation. 


* Because there are two memory mapped ports in the DMA we will have to **Run Block Automation** again. This process will now connect the DMA *streaming to memory map* master interface (**/axi_dma_0/M_AXI_S2MM**) to the processing system (PS) high performance AXI slave interface (**S_AXI_HP0**). Click **OK**. The next figure shows the block design with the AXI streaming connections highlighted.

![Figure](../lab10/lab10_figures/fig3.png)


* Note that the processor needs to have a general purpose AXI interface with the DMA to transfer control information. The next figure shows the block design with the control AXI channel highlighted.

![Figure](../lab10/lab10_figures/fig4.png)


* We will now rename the IP modules because we are gonna have to refer to these names in Python:
    * Rename the AXI DMA (axi_dma_0) as **fir_dma**
    * Rename the FIR Compiler (fir_compiler_0) as **fir**.


* To make things nicer in Python, select both the **fir** and **fir_dma** blocks (with the *CTRL* key), righ-click and select **Create Hierarchy** to bring these two IPs togetther into a hierarchy. Specify the cell name as **filter**. This will basically group them together into one block.


<div class="alert alert-block alert-info">
<b>Note:</b> In Python we are going to be referring to everything through this filter hierarchy. So this makes things a little bit more ordered in Python code.
</div>

![Figure](../lab10/lab10_figures/fig5.png)


* **Save** the block design, **Create a HDL wrapper** and let Vivado manage it.


* Once finished, click **Generate Bitstream**. When you generate the bitstream, both the **BIT** and the **HWH** files will be generated, and these will be necessary to define the overlay for PYNQ. Click **Cancel** with the generate bitstream completes.


* A **TCL** file must be generated and provided with the bitstream file as part of an overlay. To generate the TCL for the Block Diagram from the Vivado GUI, go to **File -> Export -> Export Block Design**. Rename the TCL file as **fir_accel** and click **OK**.


<div class="alert alert-block alert-info">
<b>Note:</b> The Tcl from the Vivado IP Integrator block design for the PL design is used by PYNQ to automatically identify the Zynq system configuration, IP including versions, interrupts, resets, and other control signals. Based on this information, some parts of the system configuration can be automatically modified from PYNQ, drivers can be automatically assigned, features can be enabled or disabled, and signals can be connected to corresponding Python methods.
</div>


* Now we have to copy the **TCL**, **BIT** and **HWH** files, assign them the same name (**fir_accel**), put them together and send them over to the PYNQ-Z2 board:
    * The **TCL** should be located in the project main folder.
    * The **BIT** file can be found under the **lab10.runs /impl_1** folder.
    * The **HWH** file can be found under the **lab10.srcs /sources_1 /bd /lab10bd /hw_handoff** folder.


* Copy these files into the PYNQ folder **pynq/overlays/fir_accel**.  Send files to PYNQ vis sftp: from the folder with the 3 files launch terminal and write (using your board's IP):  
`sftp xilinx@192.168.1.50 | go to pynq/overlays | mkdir fir_accel | put fir_accel.bit `


* Repeat the previous instruction until the 3 files are uploaded.

<div class="alert alert-block alert-warning">
<b>Warning:</b> If SFTP is not recogized in your system you should instal FileZilla to upload the files to the board, as shown in the figure below. You can also upload the files to your Jupyter notebook folder, as you did in previous labs!
</div>


![Figure](../lab10/lab10_figures/fig6.png)


Now the overlay is defined and ready to use!

### Step 1.3 ###

Now that the overlay is done we can continue and benchmark the hardware filter function. 

* Load the **fir_accell** overlay, import the DMA driver and load the AXI DMA object.

In [ ]:
from pynq import Overlay
import pynq.lib.dma

# Load the overlay
overlay = Overlay('/home/xilinx/pynq/overlays/fir_accel/fir_accel.bit')

# Load the FIR DMA
dma = overlay.filter.fir_dma

* The next block of code will set up the DMA transfer, so it's going to transfer my samples into the FIR filter and receive what comes out of the FIR filter. It's all going to go from memory through the filter and back into the memory. We can see it took **3ms**, so we did get an acceleration of that algorithm (**17x**).

In [ ]:
from pynq import allocate
import numpy as np

# Allocate buffers for the input and output signals
in_buffer = allocate(shape=(n,), dtype=np.int32)
out_buffer = allocate(shape=(n,), dtype=np.int32)

# Copy the samples to the in_buffer
np.copyto(in_buffer,samples)

# Trigger the DMA transfer and wait for the result
import time
start_time = time.time()
dma.sendchannel.transfer(in_buffer)
dma.recvchannel.transfer(out_buffer)
dma.sendchannel.wait()
dma.recvchannel.wait()
stop_time = time.time()
hw_exec_time = stop_time-start_time
print('Hardware FIR execution time: ',hw_exec_time)
print('Hardware acceleration factor: ',sw_exec_time / hw_exec_time)

# Plot to the notebook
plot_to_notebook(t,samples,1000,out_signal=out_buffer)

# Free the buffers
in_buffer.close()
out_buffer.close()

* To hide the details of the DMA from the programmer, and make the accelerator easier to use, we create a driver for it. Below is the driver that will be associated with the filter hierarchy that we created in Vivado, consisting of the FIR IP and the DMA.

In [ ]:
from pynq import DefaultHierarchy
from pynq import allocate

class FirDriver(DefaultHierarchy):
    def __init__(self, description):
        super().__init__(description)

    def fir_filter(self, data):
        with allocate(shape=(len(data),), dtype=np.int32) as in_buffer,\
             allocate(shape=(len(data),), dtype=np.int32) as out_buffer:
            np.copyto(in_buffer,data)
            self.fir_dma.sendchannel.transfer(in_buffer)
            self.fir_dma.recvchannel.transfer(out_buffer)
            self.fir_dma.sendchannel.wait()
            self.fir_dma.recvchannel.wait()
            result = out_buffer.copy()
        return result

    @staticmethod
    def checkhierarchy(description):
        if 'fir_dma' in description['ip']:
            return True
        else:
            return False

* We need to reload the overlay in order for the driver to be attached to the accelerator.

In [ ]:
from pynq import Overlay
import pynq.lib.dma

overlay = Overlay('/home/xilinx/pynq/overlays/fir_accel/fir_accel.bit')

# Run the hardware FIR and measure the runtime
start_time = time.time()
result = overlay.filter.fir_filter(samples)
stop_time = time.time()
print('Hardware FIR execution time (with driver): ',stop_time-start_time)

# Plot to the notebook
plot_to_notebook(t,samples,1000,out_signal=out_buffer)

We see that the result is the same but it takes much longer when using the software driver!

____________